<a href="https://colab.research.google.com/github/singh1317/trial1/blob/main/rbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Boltzmann Machines**

# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import re

#Importing the dataset

In [8]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, names=['movie_id', 'movie_name', 'genre'], engine = 'python', encoding = 'latin-1')
display(movies)
users = pd.read_csv('users.dat', sep = '::', header = None, names=['user_id', 'gender', 'age', 'job_code', 'zip_code'], engine = 'python', encoding = 'latin-1')
display(users)
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, names=['user_id', 'movie_id', 'rating', 'timestamp'], engine = 'python', encoding = 'latin-1')
display(ratings)

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


,user_id,gender,age,job_code,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
218503,1325,3035,4,974778325
218504,1325,3036,4,974776814
218505,1325,900,4,974778676
218506,1325,2092,3,974777412


# Preparing the training set and the test set

In [9]:
training_set = pd.read_csv('u1.base', delimiter = '\t', header = None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
display(training_set)
training_set = np.array(training_set, dtype = 'int')


test_set = pd.read_csv('u1.test', delimiter = '\t', header = None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
display(test_set)
test_set = np.array(test_set, dtype = 'int')

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


,user_id,movie_id,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
...,...,...,...,...
19995,458,648,4,886395899
19996,458,1101,4,886397931
19997,459,934,3,879563639
19998,460,10,3,882912371


# Getting the number of users and movies

In [10]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
print('Number of Users:', nb_users)
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print('Number of Movies:', nb_movies)

Number of Users: 943
Number of Movies: 1682


# Converting the data into an array with users in lines and movies in columns

In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
#display(training_set)
test_set = convert(test_set)
#display(test_set)

# Converting the data into Torch tensors

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [13]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the architecture of the Neural Network

In [14]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Training the RBM

In [15]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3545)
epoch: 2 loss: tensor(0.2566)
epoch: 3 loss: tensor(0.2499)
epoch: 4 loss: tensor(0.2476)
epoch: 5 loss: tensor(0.2492)
epoch: 6 loss: tensor(0.2452)
epoch: 7 loss: tensor(0.2470)
epoch: 8 loss: tensor(0.2479)
epoch: 9 loss: tensor(0.2458)
epoch: 10 loss: tensor(0.2478)


# Testing the RBM

In [16]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2602)


# Getting List of Recommended Movies for an User

In [17]:
def recommend_movies_for_user(user_id, rbm, num_recommendations=5):
    # Ensure user_id is a Python integer
    user_id = int(user_id)

    # Get the user's ratings from the training set and convert to integers
    user_ratings = training_set[user_id - 1].numpy().astype(int)

    # Create an input tensor for the RBM
    v = torch.FloatTensor(user_ratings).unsqueeze(0)

    # Sample hidden units given the user's ratings
    _, h = rbm.sample_h(v)

    # Sample visible units (movie ratings) based on hidden units
    _, recommended_ratings = rbm.sample_v(h)

    # Ensure both original_ratings and recommended_ratings have the same shape
    if recommended_ratings.shape != user_ratings.shape:
        recommended_ratings = recommended_ratings[:, :user_ratings.shape[0]]  # Trim if needed

    # Get the original movie ratings for the user
    original_ratings = test_set[user_id - 1].numpy().astype(int)

    # Replace original ratings with -1 (indicating unseen movies)
    recommended_ratings[:, original_ratings >= 0] = -1

    # Get the indices of the top N recommended movies
    top_indices = (-recommended_ratings).argsort()[0, :][:num_recommendations]

    # Get the movie names for the recommended indices

    recommended_movies = []
    for i in top_indices:
      recommended_movies.append([int(movies['movie_id'][movies['movie_id']==int(i)]), re.search(r'([A-Za-z\s\',.!?&-]+ \(\d{4}\))', str(movies['movie_name'][movies['movie_id']==int(i)]))[1], re.search(r'\s+(.+)\s+Name: genre', str(movies['genre'][movies['movie_id']==int(i)]))[1]])
    recommended_movies = pd.DataFrame(recommended_movies, columns=['movie_id', 'movie_name', 'genre'])

    return recommended_movies


In [18]:
user_id = 1 # Replace with the user ID for which you want recommendations
num_recommendations=5
recommended_movies = recommend_movies_for_user(user_id, rbm, num_recommendations)

print("Top 5 Recommended Movies for user_id:", user_id)
display(recommended_movies)


already_rated_movies = []
for i in ratings['movie_id'][ratings['user_id']==10]:
  try:
    already_rated_movies.append([int(movies['movie_id'][movies['movie_id']==int(i)]), re.search(r'([A-Za-z\s\',.!?&-]+ \(\d{4}\))', str(movies['movie_name'][movies['movie_id']==int(i)]))[1], re.search(r'\s+(.+)\s+Name: genre', str(movies['genre'][movies['movie_id']==int(i)]))[1]])
  except:
    continue

already_rated_movies = pd.DataFrame(already_rated_movies, columns=['movie_id', 'movie_name', 'genre'])

print("Already Rated Movies by user_id:", user_id)
display(already_rated_movies)

Top 5 Recommended Movies for user_id: 1


<ipython-input-17-863c2b3c4f45>:34: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  recommended_movies.append([int(movies['movie_id'][movies['movie_id']==int(i)]), re.search(r'([A-Za-z\s\',.!?&-]+ \(\d{4}\))', str(movies['movie_name'][movies['movie_id']==int(i)]))[1], re.search(r'\s+(.+)\s+Name: genre', str(movies['genre'][movies['movie_id']==int(i)]))[1]])


,movie_id,movie_name,genre
0,931,Spellbound (1945),Mystery|Romance|Thriller
1,944,Lost Horizon (1937),Drama
2,943,"Ghost and Mrs. Muir, The (1947)",Drama|Romance
3,941,"Mark of Zorro, The (1940)",Adventure
4,938,Gigi (1958),Musical


<ipython-input-18-ba422792dd8b>:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  already_rated_movies.append([int(movies['movie_id'][movies['movie_id']==int(i)]), re.search(r'([A-Za-z\s\',.!?&-]+ \(\d{4}\))', str(movies['movie_name'][movies['movie_id']==int(i)]))[1], re.search(r'\s+(.+)\s+Name: genre', str(movies['genre'][movies['movie_id']==int(i)]))[1]])


Already Rated Movies by user_id: 1


,movie_id,movie_name,genre
0,2622,"Midsummer Night's Dream, A (1999)",Comedy|Fantasy
1,648,Impossible (1996),Action|Adventure|Mystery
2,2628,Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
3,3358,Defending Your Life (1991),Comedy|Romance
4,3359,Breaking Away (1979),Drama
...,...,...,...
376,2045,"Far Off Place, A (1993)",Adventure|Children's|Drama|Romance
377,2046,Flight of the Navigator (1986),Adventure|Children's|Sci-Fi
378,2047,"Gnome-Mobile, The (1967)",Children's
379,1247,"Graduate, The (1967)",Drama|Romance
